<a href="https://colab.research.google.com/github/Daniel-Karlik/Parliament_Spotlight_NLP/blob/Data/Data_clean_%2B_transform_to_pickle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Funkce pro transformaci a čištění dat 

In [ ]:
# All needed packages
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import glob
import re
import string
from google.colab import drive

In [ ]:
# Mounting google drive
# drive.mount('/content/drive/')
# # Path to our data on google drive
# DATA_PATH = "/content/drive/MyDrive/NLP/"
drive.mount('/content/drive/')
# Path to our data on google drive
DATA_PATH = "/content/drive/MyDrive/NLP/"
# Creating file_list containing all .pickle files
file_list = glob.glob(DATA_PATH + "/*.pickle")

Mounted at /content/drive/


In [ ]:
print(file_list[0])

/content/drive/MyDrive/NLP/stenozaznamy_psp_month_2022_10.pickle


In [ ]:
infile = open(file_list[0],'rb')
file = pickle.load(infile)
#main_dataframe = pd.DataFrame.from_dict(file_list[0]["results"])

In [ ]:
type(file)

dict

Here we use function 

```
pd.DataFrame.from_dict
```
which transforms dictionary into pd.DataFrame


In [ ]:
main_dataframe = pd.DataFrame.from_dict(file["results"])
#main_dataframe.head()

Now, let's import and merge all our data into one DataFrame 

In [ ]:
for i in range(1,len(file_list)):
  infile = open(file_list[i],'rb')
  file = pickle.load(infile)
  data = pd.DataFrame.from_dict(file["results"])
  df = pd.DataFrame(data)
  main_dataframe = pd.concat([main_dataframe,df],axis=0)
print(main_dataframe.describe())

              poradi         obdobi         schuze      pocetSlov
count  410104.000000  410104.000000  410104.000000  410104.000000
mean     1114.542055    2008.475077      35.490939     144.724506
std       912.881045       7.091470      21.314870     306.245896
min         0.000000    1998.000000       1.000000       1.000000
25%       359.000000    2002.000000      20.000000      19.000000
50%       928.000000    2010.000000      33.000000      43.000000
75%      1672.000000    2013.000000      48.000000     148.000000
max      6121.000000    2021.000000     120.000000   18977.000000


Now we need to use only usefull data, so we set our threshold for filtering only the useful text data.

## Selected threshold is 300!

In [ ]:
threshold = 300

In [ ]:
df_cut = main_dataframe[main_dataframe['pocetSlov'] > threshold]

In [ ]:
df_cut.head()

,Id,poradi,obdobi,datum,schuze,url,cisloHlasovani,celeJmeno,narozeni,HsProcessType,OsobaId,funkce,tema,text,pocetSlov,politiciZminky,temata,DbCreated,DbCreatedBy
0,2021_42_00010,10,2021,2022-10-18T00:00:00+02:00,42,http://www.psp.cz/eknih/2021ps/stenprot/042sch...,NaN,Tomio Okamura,None,person,tomio-okamura,Poslanec,Zahájení schůze,"Děkuji. Vše, co říkám, není samozřejmě nic nov...",2818,"[petr-fiala-111, vit-rakusan, jan-hrncir, jiri...",None,2022-10-18T23:00:12.0983703Z,None
1,2021_42_00012,12,2021,2022-10-18T00:00:00+02:00,42,http://www.psp.cz/eknih/2021ps/stenprot/042sch...,NaN,Alena Schillerová,None,person,alena-schillerova,Poslankyně,Zahájení schůze,"Děkuji za slovo, paní místopředsedkyně. Dámy a...",2333,"[petr-fiala-111, karel-havlicek-3]",None,2022-10-18T23:00:17.9044387Z,None
12,2021_42_00042,42,2021,2022-10-18T00:00:00+02:00,42,http://www.psp.cz/eknih/2021ps/stenprot/042sch...,NaN,Karel Havlíček,None,person,karel-havlicek-3,Místopředseda PSP,Zahájení schůze,"Děkuji, vážený pane předsedající, vážený pane ...",2486,None,None,2022-10-18T23:00:43.2893255Z,None
13,2021_42_00044,44,2021,2022-10-18T00:00:00+02:00,42,http://www.psp.cz/eknih/2021ps/stenprot/042sch...,NaN,Marek Novák,None,person,marek-novak-12,Poslanec,Zahájení schůze,"Děkuji za slovo, pane předsedající. Vážené kol...",1162,"[marketa-pekarova-adamova, miroslav-kalousek]",None,2022-10-18T23:00:46.843141Z,None
14,2021_42_00046,46,2021,2022-10-18T00:00:00+02:00,42,http://www.psp.cz/eknih/2021ps/stenprot/042sch...,NaN,Jana Mračková Vildumetzová,None,person,jana-mrackova-vildumetzova,Poslankyně,Zahájení schůze,Vážený pane místopředsedo (místopředseda Barto...,1264,"[jan-bartosek, hubert-lang, drahoslav-ryba]",None,2022-10-18T23:00:50.1025816Z,None


Here we can see, that there are too many useless columns, and indices are not as expected (now there are even rows with same indices, because they were used from all merged files).

In [ ]:
# Dropping useless columns from dataframe
df_cut = df_cut.drop(['poradi', 'url', 'cisloHlasovani', 'narozeni', 'HsProcessType', 'temata', 'DbCreated', 'DbCreatedBy'], axis=1)

In [ ]:
# Here we reindex all remaining data
df_cut = df_cut.reset_index()

In [ ]:
df_cut.describe(include='all')

,index,Id,obdobi,datum,schuze,celeJmeno,OsobaId,funkce,tema,text,pocetSlov,politiciZminky
count,49083.000000,49083,49083.000000,49083,49083.000000,49083,47778,49083,49083,49083,49083.000000,47479
unique,NaN,48962,NaN,1641,NaN,1177,1040,124,1272,48949,NaN,11760
top,NaN,2021_42_00010,NaN,2011-11-02T00:00:00+01:00,NaN,Zbyněk Stanjura,zbynek-stanjura,Poslanec,Zahájení schůze,"Vážený pane místopředsedo, vážený pane ministř...",NaN,[]
freq,NaN,2,NaN,109,NaN,1273,1356,25976,13479,3,NaN,23326
mean,1030.950044,NaN,2008.956624,NaN,34.951816,NaN,NaN,NaN,NaN,NaN,717.637838,NaN
std,745.660727,NaN,7.165667,NaN,21.481419,NaN,NaN,NaN,NaN,NaN,611.010222,NaN
min,0.000000,NaN,1998.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,301.000000,NaN
25%,410.000000,NaN,2002.000000,NaN,19.000000,NaN,NaN,NaN,NaN,NaN,383.000000,NaN
50%,903.000000,NaN,2010.000000,NaN,33.000000,NaN,NaN,NaN,NaN,NaN,531.000000,NaN
75%,1536.000000,NaN,2017.000000,NaN,47.000000,NaN,NaN,NaN,NaN,NaN,812.000000,NaN


We can see that 'tema' column has 1272 unique values, so we keep it, it could be useful.
But in the column 'text', which is our main point of interest we can see, that not all 'text' values are unique, so we drop those which are duplicated, something around 140 values. 

In [ ]:
df_cut = df_cut.drop_duplicates(subset=['text'])

In [ ]:
df_cut.head()

,Id,obdobi,datum,schuze,celeJmeno,OsobaId,funkce,tema,text,pocetSlov,politiciZminky
0,2021_42_00010,2021,2022-10-18T00:00:00+02:00,42,Tomio Okamura,tomio-okamura,Poslanec,Zahájení schůze,"Děkuji. Vše, co říkám, není samozřejmě nic nov...",2818,"[petr-fiala-111, vit-rakusan, jan-hrncir, jiri..."
1,2021_42_00012,2021,2022-10-18T00:00:00+02:00,42,Alena Schillerová,alena-schillerova,Poslankyně,Zahájení schůze,"Děkuji za slovo, paní místopředsedkyně. Dámy a...",2333,"[petr-fiala-111, karel-havlicek-3]"
2,2021_42_00042,2021,2022-10-18T00:00:00+02:00,42,Karel Havlíček,karel-havlicek-3,Místopředseda PSP,Zahájení schůze,"Děkuji, vážený pane předsedající, vážený pane ...",2486,None
3,2021_42_00044,2021,2022-10-18T00:00:00+02:00,42,Marek Novák,marek-novak-12,Poslanec,Zahájení schůze,"Děkuji za slovo, pane předsedající. Vážené kol...",1162,"[marketa-pekarova-adamova, miroslav-kalousek]"
4,2021_42_00046,2021,2022-10-18T00:00:00+02:00,42,Jana Mračková Vildumetzová,jana-mrackova-vildumetzova,Poslankyně,Zahájení schůze,Vážený pane místopředsedo (místopředseda Barto...,1264,"[jan-bartosek, hubert-lang, drahoslav-ryba]"


In [ ]:
df_cut = df_cut.drop(['index'], axis=1)

In [ ]:
df_cut.describe(include='all')

,index,Id,obdobi,datum,schuze,url,celeJmeno,OsobaId,funkce,tema,text,pocetSlov,politiciZminky
count,48949.000000,48949,48949.000000,48949,48949.000000,48949,48949,47659,48949,48949,48949,48949.000000,47401
unique,NaN,48949,NaN,1641,NaN,38090,1177,1040,124,1272,48949,NaN,11760
top,NaN,2021_42_00010,NaN,2011-11-02T00:00:00+01:00,NaN,http://www.psp.cz/eknih/2013ps/stenprot/017sch...,Zbyněk Stanjura,zbynek-stanjura,Poslanec,Zahájení schůze,"Děkuji. Vše, co říkám, není samozřejmě nic nov...",NaN,[]
freq,NaN,1,NaN,109,NaN,4,1273,1344,25919,13395,1,NaN,23326
mean,1031.233999,NaN,2008.923737,NaN,34.953523,NaN,NaN,NaN,NaN,NaN,NaN,716.743938,NaN
std,745.992934,NaN,7.147785,NaN,21.506913,NaN,NaN,NaN,NaN,NaN,NaN,609.010229,NaN
min,0.000000,NaN,1998.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,301.000000,NaN
25%,410.000000,NaN,2002.000000,NaN,19.000000,NaN,NaN,NaN,NaN,NaN,NaN,383.000000,NaN
50%,903.000000,NaN,2010.000000,NaN,33.000000,NaN,NaN,NaN,NaN,NaN,NaN,531.000000,NaN
75%,1536.000000,NaN,2017.000000,NaN,47.000000,NaN,NaN,NaN,NaN,NaN,NaN,811.000000,NaN


This seems OK now.

## Let's focus on transformation of text values. Removing notes, punctuation and content inside brackets.

In [ ]:
def clean_text_round1(text):
    '''Remove text in round brackets, remove notes and remove punctuation.'''
    ##text = text.lower()
    pattern = 'Neprošlo jazykovou korekturou, neautorizováno !' # This part is inside many transcripts
    text = re.sub(pattern, '', text)
    text = re.sub('\(.*?\)', '', text)
    text = re.sub('\*', '', text)
    #text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    ##text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

Just one example of text with note: "*** Neprošlo jazykovou korekturou, neautorizováno !" and content inside () at the end of this text example.

In [ ]:
textik = df_cut['text']
print(textik[0])

Děkuji. Vše, co říkám, není samozřejmě nic nového. Dokonce jsme v neuvěřitelné situaci, kdy vláda dělá takové tragické chyby, že ji kritizují i členové vládních stran.***
Neprošlo jazykovou korekturou, neautorizováno !
Například exministr financí a zakladatel vládní koaliční TOP 09 Miroslav Kalousek řekl, že vláda Petra Fialy je nejméně rozpočtově odpovědná vláda a že je politicky zbabělá. To o vás říká váš koaliční kolega, člen vládní TOP 09! Také se pan Kalousek zmínil o tom, že 900 miliard výdajů nesouvisí s krizí, ani s válkou. Takže už dokonce i vaši koaliční kolegové, váš exministr financí z vaší vládní koaliční strany už o vás říká, že jste politicky zbabělí a že jste nejméně rozpočtově odpovědní. Takže už dokonce i vaši nejznámější exponenti už samozřejmě vás kritizují, protože už se na to nemůžou dívat.
Ano, tak jako u předchozích vlád byl pláštíkem pro enormní výdaje covid, tak dneska se stal zaklínadlem ministrů Putin. Pravda je ale taková, že kvůli Fialově neschopné vládě n

Output of our cleaning.

In [ ]:
textik2 = clean_text_round1(textik[0])
print(textik2)

Děkuji. Vše, co říkám, není samozřejmě nic nového. Dokonce jsme v neuvěřitelné situaci, kdy vláda dělá takové tragické chyby, že ji kritizují i členové vládních stran.

Například exministr financí a zakladatel vládní koaliční TOP 09 Miroslav Kalousek řekl, že vláda Petra Fialy je nejméně rozpočtově odpovědná vláda a že je politicky zbabělá. To o vás říká váš koaliční kolega, člen vládní TOP 09! Také se pan Kalousek zmínil o tom, že 900 miliard výdajů nesouvisí s krizí, ani s válkou. Takže už dokonce i vaši koaliční kolegové, váš exministr financí z vaší vládní koaliční strany už o vás říká, že jste politicky zbabělí a že jste nejméně rozpočtově odpovědní. Takže už dokonce i vaši nejznámější exponenti už samozřejmě vás kritizují, protože už se na to nemůžou dívat.
Ano, tak jako u předchozích vlád byl pláštíkem pro enormní výdaje covid, tak dneska se stal zaklínadlem ministrů Putin. Pravda je ale taková, že kvůli Fialově neschopné vládě nakupujeme předražené energie přes spekulanty na li

In [ ]:
df_cut['text'] = df_cut['text'].apply(lambda x: clean_text_round1(x))
df_cut.head()

,Id,obdobi,datum,schuze,celeJmeno,OsobaId,funkce,tema,text,pocetSlov,politiciZminky
0,2021_42_00010,2021,2022-10-18T00:00:00+02:00,42,Tomio Okamura,tomio-okamura,Poslanec,Zahájení schůze,"Děkuji. Vše, co říkám, není samozřejmě nic nov...",2818,"[petr-fiala-111, vit-rakusan, jan-hrncir, jiri..."
1,2021_42_00012,2021,2022-10-18T00:00:00+02:00,42,Alena Schillerová,alena-schillerova,Poslankyně,Zahájení schůze,"Děkuji za slovo, paní místopředsedkyně. Dámy a...",2333,"[petr-fiala-111, karel-havlicek-3]"
2,2021_42_00042,2021,2022-10-18T00:00:00+02:00,42,Karel Havlíček,karel-havlicek-3,Místopředseda PSP,Zahájení schůze,"Děkuji, vážený pane předsedající, vážený pane ...",2486,None
3,2021_42_00044,2021,2022-10-18T00:00:00+02:00,42,Marek Novák,marek-novak-12,Poslanec,Zahájení schůze,"Děkuji za slovo, pane předsedající. Vážené kol...",1162,"[marketa-pekarova-adamova, miroslav-kalousek]"
4,2021_42_00046,2021,2022-10-18T00:00:00+02:00,42,Jana Mračková Vildumetzová,jana-mrackova-vildumetzova,Poslankyně,Zahájení schůze,"Vážený pane místopředsedo , vážený pane minist...",1264,"[jan-bartosek, hubert-lang, drahoslav-ryba]"


In [ ]:
df_cut.describe(include='all')

,index,Id,obdobi,datum,schuze,url,celeJmeno,OsobaId,funkce,tema,text,pocetSlov,politiciZminky
count,48949.000000,48949,48949.000000,48949,48949.000000,48949,48949,47659,48949,48949,48949,48949.000000,47401
unique,NaN,48949,NaN,1641,NaN,38090,1177,1040,124,1272,48949,NaN,11760
top,NaN,2021_42_00010,NaN,2011-11-02T00:00:00+01:00,NaN,http://www.psp.cz/eknih/2013ps/stenprot/017sch...,Zbyněk Stanjura,zbynek-stanjura,Poslanec,Zahájení schůze,Děkuji Vše co říkám není samozřejmě nic nového...,NaN,[]
freq,NaN,1,NaN,109,NaN,4,1273,1344,25919,13395,1,NaN,23326
mean,1031.233999,NaN,2008.923737,NaN,34.953523,NaN,NaN,NaN,NaN,NaN,NaN,716.743938,NaN
std,745.992934,NaN,7.147785,NaN,21.506913,NaN,NaN,NaN,NaN,NaN,NaN,609.010229,NaN
min,0.000000,NaN,1998.000000,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,301.000000,NaN
25%,410.000000,NaN,2002.000000,NaN,19.000000,NaN,NaN,NaN,NaN,NaN,NaN,383.000000,NaN
50%,903.000000,NaN,2010.000000,NaN,33.000000,NaN,NaN,NaN,NaN,NaN,NaN,531.000000,NaN
75%,1536.000000,NaN,2017.000000,NaN,47.000000,NaN,NaN,NaN,NaN,NaN,NaN,811.000000,NaN


In [ ]:
df_cut.to_pickle("./psp_data_300.pkl")  